# 1트 구조

In [12]:
# disease_retrieval.py
import pandas as pd
import glob
import os
import json

def load_disease_data(path):
    """
    지정된 경로가 파일이면 해당 JSON 파일을 읽어 DataFrame으로 변환,
    경로가 디렉토리이면 그 안의 모든 JSON 파일을 읽어 결합한 DataFrame 반환

    JSON 파일은 아래와 같은 구조의 리스트 형태여야 함:
      [
        {
          "disease": "구강건조증",
          "section": "개요",
          "content": "구강건조증은 ~~~"
        },
        ...
      ]
    """
    data_list = []
    
    if os.path.isdir(path):
        # 경로가 디렉토리인 경우, 디렉토리 내의 모든 JSON 파일 검색
        json_files = glob.glob(os.path.join(path, "*.json"))
        if not json_files:
            raise FileNotFoundError(f"{path} 폴더 내에 JSON 파일이 없습니다.")
        for json_file in json_files:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)
                if isinstance(data, list):
                    data_list.extend(data)
                else:
                    raise ValueError(f"{json_file} 파일의 최상위 구조는 리스트여야 합니다.")
    else:
        # 경로가 파일인 경우
        if not os.path.exists(path):
            raise FileNotFoundError(f"{path} 파일이 존재하지 않습니다.")
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            data_list = data
        else:
            raise ValueError("JSON 파일의 최상위 구조는 리스트여야 합니다.")
    
    return pd.DataFrame(data_list)

def retrieve_disease_info(disease_data, user_symptoms):
    """
    사용자 증상과 질병 데이터 간의 유사도 기반 retrieval 수행
    현재는 retrieval 모델이 별도로 있다고 가정하고, 예시로 첫 번째 행을 반환함
    실제 구현에서는 임베딩 및 코사인 유사도 계산 등으로 가장 유사한 질병 정보를 반환할 것임
    """
    # TODO: 임베딩 및 유사도 계산 로직 추가 (현재는 예시로 첫 행 선택)
    retrieved_row = disease_data.iloc[0]
    return retrieved_row.to_dict()

def generate_prompt(user_symptoms, retrieved_info):
    """
    사용자 증상과 retrieval 결과를 결합해 생성 모델에 입력할 프롬프트 템플릿 구성
    """
    prompt = f"환자의 증상: {user_symptoms}\n"
    prompt += "관련 질병 정보:\n"
    prompt += f"- 질병명: {retrieved_info.get('Disease', '정보 없음')}\n"
    prompt += f"- 증상: {retrieved_info.get('Symptoms', '정보 없음')}\n"
    prompt += f"- 추천 진료과: {retrieved_info.get('Department', '정보 없음')}\n"
    prompt += f"- 추천 의사: {retrieved_info.get('Doctor', '정보 없음')} (특화 질병: {retrieved_info.get('Specialization', '정보 없음')})\n"
    prompt += "위 정보를 토대로, 왜 해당 질병이 매칭되었는지와 추가 의료 조언을 상세히 설명해줘."
    return prompt



In [13]:
def main():
    # 1. 사용자 증상 입력 (인풋 값)
    user_symptoms = "나 요즘 구강에 건조함을 느끼고, 마른 음식을 먹을 때 물이 필요하다고 느껴."
    
    # 2. 질병 데이터 불러오기 (CSV 파일들이 'data' 폴더에 있다고 가정)
    data_folder = "data"
    try:
        df = load_disease_data(data_folder)
    except FileNotFoundError as e:
        print(e)
        return
    
    # 3. Retrieval: 사용자 증상과 유사한 질병 정보 선택 (현재는 단순 예시)
    retrieved_info = retrieve_disease_info(df, user_symptoms)
    
    # 4. 프롬프트 생성: Generator 모듈에 전달할 프롬프트 구성
    prompt = generate_prompt(user_symptoms, retrieved_info)
    
    # 5. 최종 프롬프트 출력 (이후 generator 모듈로 전달해 생성 작업 진행)
    print("=== 생성된 프롬프트 ===")
    print(prompt)

if __name__ == "__main__":
    main()


=== 생성된 프롬프트 ===
환자의 증상: 나 요즘 구강에 건조함을 느끼고, 마른 음식을 먹을 때 물이 필요하다고 느껴.
관련 질병 정보:
- 질병명: 정보 없음
- 증상: 정보 없음
- 추천 진료과: 정보 없음
- 추천 의사: 정보 없음 (특화 질병: 정보 없음)
위 정보를 토대로, 왜 해당 질병이 매칭되었는지와 추가 의료 조언을 상세히 설명해줘.


# 2트 구조 추가

In [16]:
import os
import glob
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def load_disease_data(path):
    """
    지정된 경로가 파일이면 해당 JSON 파일을 읽어 DataFrame으로 변환,
    경로가 디렉토리이면 그 안의 모든 JSON 파일을 읽어 결합한 DataFrame 반환
    
    JSON 파일은 아래와 같은 구조의 리스트 형태여야 함:
      [
        {
          "disease": "구강건조증",
          "section": "개요",
          "content": "구강건조증은 ~~~"
        },
        {
          "disease": "구강건조증",
          "section": "증상",
          "content": "구강건조증이나 타액분비저하증을 진단하려면 ~~~"
        },
        ...
      ]
    """
    data_list = []
    
    if os.path.isdir(path):
        json_files = glob.glob(os.path.join(path, "*.json"))
        if not json_files:
            raise FileNotFoundError(f"{path} 폴더 내에 JSON 파일이 없습니다.")
        for json_file in json_files:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)
                if isinstance(data, list):
                    data_list.extend(data)
                else:
                    raise ValueError(f"{json_file} 파일의 최상위 구조는 리스트여야 합니다.")
    else:
        if not os.path.exists(path):
            raise FileNotFoundError(f"{path} 파일이 존재하지 않습니다.")
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            data_list = data
        else:
            raise ValueError("JSON 파일의 최상위 구조는 리스트여야 합니다.")
    
    return pd.DataFrame(data_list)

def retrieve_disease_info(disease_data, user_symptoms):
    """
    사용자 증상과 매칭하기 위해 '증상' 섹션에 해당하는 데이터를 검색하여 retrieval 수행
    실제 임베딩 기반 유사도 계산을 할 수 있지만, 여기서는 예시로 '증상' 섹션의 첫 번째 행을 선택함
    """
    symptom_rows = disease_data[disease_data['section'] == '증상']
    if symptom_rows.empty:
        raise ValueError("증상 정보가 포함된 데이터가 없습니다.")
    # TODO: 실제 임베딩 및 유사도 계산 로직 추가 가능
    retrieved_row = symptom_rows.iloc[0]
    # 반환 값에는 원본 텍스트(content)와 질병명 등 필요한 정보 포함
    return retrieved_row.to_dict()

def generate_prompt(user_symptoms, retrieved_info):
    """
    사용자 증상과 retrieval된 '증상' 정보를 결합해 생성 모델에 입력할 프롬프트를 구성함.
    여기서는 retrieval된 정보의 'content' 필드(원본 증상 텍스트)를 활용함.
    """
    prompt = f"환자의 증상: {user_symptoms}\n"
    prompt += "관련 질병 증상 정보:\n"
    prompt += f"{retrieved_info.get('content', '정보 없음')}\n"
    prompt += "위 정보를 토대로, 왜 해당 질병이 매칭되었는지와 추가 의료 조언을 상세히 설명해줘."
    return prompt

def generate_medical_explanation(prompt, generator_pipeline):
    """
    생성기(generator) 모듈: 구성된 프롬프트를 입력받아 LLM(GPT)을 통해 최종 결과를 생성함.
    """
    # 디코딩 전략은 max_length, num_beams 등으로 조정할 수 있음.
    result = generator_pipeline(prompt, max_length=256, num_return_sequences=1)
    return result[0]['generated_text']




In [20]:
def main():
    # 1. 데이터 로드: './data' 폴더에 JSON 파일(예: "구강건조증_realjson.json")들이 있다고 가정
    data_path = "data"
    try:
        disease_data = load_disease_data(data_path)
        print("데이터 불러오기 성공!")
        print()
        print("-"*20)
        print()
        print(disease_data.head())
    except Exception as e:
        print(e)
        return

    # 2. 사용자 증상 입력 (예시)
    user_symptoms = "나 요즘 구강에 건조함을 느끼고, 마른 음식을 먹을 때 물이 필요하다고 느껴."
    
    # 3. Retrieval: '증상' 섹션에 해당하는 질병 정보를 가져옴
    try:
        retrieved_info = retrieve_disease_info(disease_data, user_symptoms)
    except Exception as e:
        print(e)
        return

    # 4. 프롬프트 생성: 사용자 증상과 retrieval된 원본 텍스트를 결합
    prompt = generate_prompt(user_symptoms, retrieved_info)
    print("생성할 프롬프트:")
    print(prompt)
    
    print()
    print("-"*20)
    print()
    
    # 5. Huggingface의 GPT 모델을 사용한 Generator 모듈 설정
    # 예시로 gpt2 모델 사용 (실제 서비스에서는 도메인 특화 파인튜닝 모델 고려)
    generator_pipeline = pipeline("text2text-generation", model="t5-small", tokenizer="t5-small")
    
    # 6. 생성기 모듈을 통해 최종 결과 생성
    generated_text = generate_medical_explanation(prompt, generator_pipeline)
    print("\n생성된 결과:")
    print(generated_text)

if __name__ == "__main__":
    main()

데이터 불러오기 성공!

--------------------

  disease                     section  \
0   구강건조증  건강정보 건강통계 의료기관정보 알림정보 소개마당   
1   구강건조증                       구강건조증   
2   구강건조증              • 콘텐츠명 : 구강건조증   
3   구강건조증                          개요   
4   구강건조증                      건강정보검색   

                                             content  
0                                      건강정보  건강정보   
1                                     건강담기 수정ᆞ문의    
2  등록일자 : 2021-04-21 업데이트 : 2021-11-12 조회 : 26316...  
3  구강건조증(xerostomia)은 입이 마른다고 느끼는 주관적인 증상을 의미합니다....  
4  타액선, 즉 침샘은 이하선, 악하선, 설하선 및 소타액선으로 구성됩니다. 이하선(귀...  
생성할 프롬프트:

--------------------

환자의 증상: 나 요즘 구강에 건조함을 느끼고, 마른 음식을 먹을 때 물이 필요하다고 느껴.
관련 질병 증상 정보:
구강건조증이나 타액분비저하증을 진단하려면 체계적인 접근이 중요합니다. 먼저 주관적인 증상에 관심을 기울이고 의학적으로 상세히 확인해야 합니다. 가령식사 중 구강의 건조함을 느끼거나, 마른 음식을 먹을 때 물이 반드시 필요한 경우, 연하 시 불편감을 느끼는 경우에는 실제 타액분비가 저하되었을 가능성이높습니다. 그러나 객관적인 타액분비 저하가 없어도 환자는 구강건조증의 불편감을 느낄 수 있습니다. 말을 하는 데 불편하거나 미각의 변화, 입안의 작열감, 구취등의 증상이 유발될 수 있으며, 의치를 조작하는 데 어려움을 

Device set to use mps:0



생성된 결과:
                                                  .     
